In [1]:
import os
import time
import shutil
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import keras
import gc
from PIL import Image
%matplotlib inline
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications import ResNet50, ResNet50V2, InceptionResNetV2, InceptionV3
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, cohen_kappa_score

In [2]:
tf.random.set_seed(42)

In [3]:
img_size = (768, 768)
# optimizer = SGD(learning_rate=5e-5, momentum=0.9)
optimizer = Adam(learning_rate=1e-4)
foldId = 0

In [4]:
train_datagen = ImageDataGenerator(rotation_range=90,
                            brightness_range=[0.7, 1.3],
                            zoom_range=[0.5,1.0],
                            horizontal_flip=True,
                            vertical_flip=True)
train_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabelsWithGeneratedImages.csv")

indexes = []
for i in range(len(train_df['image'])):
    if train_df['kfold'][i] == foldId:
        indexes.append(i)
train_df.drop(indexes, axis=0, inplace=True)

train_df['image'] = [str(x)+'.jpeg' for x in train_df['image']]
train_df['level'] = [str(x) for x in train_df['level']]
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/train/train",
    x_col="image",
    y_col='level',
    subset="training",
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 54710 validated image filenames belonging to 5 classes.


In [5]:
val_datagen = ImageDataGenerator()
val_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabelsWithGeneratedImages.csv")
indexes = []
for i in range(len(val_df['image'])):
    if val_df['kfold'][i] != foldId:
        indexes.append(i)
val_df.drop(indexes, axis=0, inplace=True)

val_df['image'] = [str(x)+'.jpeg' for x in val_df['image']]
val_df['level'] = [str(x) for x in val_df['level']]
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/train/train",
    x_col="image",
    y_col='level',
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 18237 validated image filenames belonging to 5 classes.


In [6]:
test_datagen = ImageDataGenerator()
test_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/Data/Data/test_labels.csv")
test_df['image'] = [str(x)+'.jpeg' for x in test_df['image']]
test_df['level'] = [str(x) for x in test_df['level']]
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/test",
    x_col="image",
    y_col='level',
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=img_size)

Found 53574 validated image filenames belonging to 5 classes.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [7]:
inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
output = inception.layers[-1].output
output = keras.layers.Flatten()(output)
inception = Model(inception.input, output)

model = Sequential()
model.add(inception)
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer, 
            loss='categorical_crossentropy',
            metrics=['accuracy'])

checkpoint_filepath = "./inceptionV3_best_model.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                          save_weights_only=False,
                          monitor='val_loss',
                          mode='auto',
                          save_best_only=True)

early_stopping = EarlyStopping(
                monitor="val_loss",
                patience=3,
                verbose=1,
                mode="auto",
)

model.summary()

2022-06-01 03:35:07.751066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-01 03:35:07.888633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-01 03:35:07.889380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-01 03:35:07.891307: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 991232)            21802784  
_________________________________________________________________
dropout (Dropout)            (None, 991232)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 4956165   
Total params: 26,758,949
Trainable params: 26,724,517
Non-trainable params: 34,432
_________________________________________________________________


In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [9]:
model = load_model("../input/dr-inceptionv3-best-model/inceptionV3_best_model.h5")

In [10]:
# model.fit_generator(generator=train_generator,
#                    steps_per_epoch=STEP_SIZE_TRAIN,
#                    validation_data=val_generator,
#                    validation_steps=STEP_SIZE_VALID,
#                    epochs=25,
#                    callbacks=[model_checkpoint_callback, early_stopping])

In [11]:
# model = load_model("./inceptionV3_best_model.h5")

In [12]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=STEP_SIZE_TEST,
                            verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2022-06-01 03:35:21.180606: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-01 03:35:23.150379: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


53574/53574 [==============================] - 1521s 28ms/step


In [13]:
# saving model predictions
a = np.asarray(pred)
np.savetxt("./testPredsInceptionV3.csv", a, delimiter=",")

In [14]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames

res = {filenames[i]: predictions[i] for i in range(len(filenames))}

In [15]:
from collections import Counter
P = Counter(predictions)
T = Counter(test_df['level'])
print(sorted(P.items()))
print(sorted(T.items()))

[('0', 43306), ('1', 932), ('2', 7384), ('3', 1122), ('4', 830)]
[('0', 39533), ('1', 3762), ('2', 7861), ('3', 1214), ('4', 1206)]


In [16]:
correct = 0
predLabels = []
indexes = []
for i in range(len(test_df['image'])):
    if test_df['image'][i] in res:
        predLabels.append(res[test_df['image'][i]])
        if res[test_df['image'][i]] == test_df['level'][i]:
            correct += 1
    else:
        predLabels.append("NA")
        indexes.append(i)
acc = (correct / len(test_df['level'])) * 100
print(acc)

test_df['predLevel'] = predLabels
test_df.drop(indexes, axis=0, inplace=True)
test_df.to_csv("./inceptionV3predictions.csv", index=False)

model.save('model.h5')

print(classification_report(test_df['level'], test_df['predLevel']))

print(confusion_matrix(test_df["level"], test_df["predLevel"]))

print("Test Accuracy score : ", accuracy_score(test_df['level'], test_df['predLevel']))
print("Test Weighted F1 score : ", f1_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Micro F1 score : ", f1_score(test_df['level'], test_df['predLevel'], average="micro"))
print("Test Precision score : ", precision_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Recall score : ", recall_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Cohen Kappa Score : ", cohen_kappa_score(test_df['level'], test_df['predLevel'], weights="quadratic"))

83.47767657159922
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     39532
           1       0.45      0.11      0.18      3762
           2       0.66      0.62      0.64      7860
           3       0.42      0.39      0.40      1214
           4       0.64      0.44      0.52      1206

    accuracy                           0.83     53574
   macro avg       0.61      0.51      0.53     53574
weighted avg       0.81      0.83      0.81     53574

[[38408   264   755    23    82]
 [ 2545   418   784     8     7]
 [ 2130   240  4897   458   135]
 [   66     4   603   468    73]
 [  157     6   345   165   533]]
Test Accuracy score :  0.8348079292193975
Test Weighted F1 score :  0.8118856747824839
Test Micro F1 score :  0.8348079292193976
Test Precision score :  0.8071378805098446
Test Recall score :  0.8348079292193975
Test Cohen Kappa Score :  0.7562184719164994
